In [6]:
import pandas as pd
from datetime import datetime

# URL to the player statistics file
players_stats_url = 'https://raw.githubusercontent.com/olbauday/FPL-Elo-Insights/main/data/2025-2026/playerstats.csv'

players_url = 'https://raw.githubusercontent.com/olbauday/FPL-Elo-Insights/main/data/2025-2026/players.csv'


# Load the data
players_stats_df = pd.read_csv(player_data_url)
players_df = pd.read_csv(player_url)

print("Player stats loaded successfully!")
players_stats_df.head()

print("Player information loaded!")
players_df.head()

Player stats loaded successfully!
Player information loaded!


,player_code,player_id,first_name,second_name,web_name,team_code,position
0,500040,662,Cristhian,Mosquera,Mosquera,3,Defender
1,224117,666,Viktor,Gyökeres,Gyökeres,3,Forward
2,221466,72,Marcos,Senesi Barón,Senesi,91,Defender
3,54469,73,Adam,Smith,Smith,91,Defender
4,494521,74,Adrien,Truffert,Truffert,91,Defender


In [19]:
# List of essential columns to select (with new features added)
core_features = [
    # IDs and Context
    'id', 'web_name', 'now_cost', 'selected_by_percent', 'form',

    # FPL Performance Metrics
    'minutes', 'total_points', 'bonus', 'bps',

    # Underlying Performance (Per 90)
    'expected_goals_per_90', 'expected_assists_per_90',
    'expected_goal_involvements_per_90', 'expected_goals_conceded_per_90',
    'starts_per_90', 'clean_sheets_per_90', 'saves_per_90',

    # Set Piece Threat
    'corners_and_indirect_freekicks_order', 'direct_freekicks_order', 'penalties_order',

    # --- NEW FEATURES ---
    # ICT Index
    'influence', 'creativity', 'threat', 'ict_index',

    # Player Status
    'status', 'chance_of_playing_next_round'
]

# Create the new DataFrame
selected_df = players_stats_df[core_features].copy()

# Fill missing values for set piece takers and chance of playing
# (A null value for chance_of_playing usually means 100%)
selected_df['chance_of_playing_next_round'] = selected_df['chance_of_playing_next_round'].fillna(100)
for col in ['corners_and_indirect_freekicks_order', 'direct_freekicks_order', 'penalties_order']:
    selected_df[col] = selected_df[col].fillna(0)

print("Core features selected, including ICT and player status.")
selected_df.head()

Core features selected, including ICT and player status.


,id,web_name,now_cost,selected_by_percent,form,minutes,total_points,bonus,bps,expected_goals_per_90,...,saves_per_90,corners_and_indirect_freekicks_order,direct_freekicks_order,penalties_order,influence,creativity,threat,ict_index,status,chance_of_playing_next_round
0,1,Raya,5.5,19.9,10.0,90,10,3,38,0.00,...,7.0,0.0,0.0,0.0,49.2,0.0,0.0,4.9,a,100.0
1,16,Saka,10.0,17.0,0.0,1724,127,18,508,0.36,...,0.0,1.0,4.0,1.0,606.0,842.8,830.0,227.9,a,100.0
2,33,M.Bizot,4.5,0.3,0.0,0,0,0,0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,a,100.0
3,45,Sousa,4.0,0.3,0.0,0,0,0,0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,a,100.0
4,24,Nørgaard,5.5,0.5,0.0,0,0,0,0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,d,75.0


In [20]:
# Select only the columns we need from players_df to avoid clutter
player_info_df = players_df[['player_id', 'position', 'team_code']]

# Merge the two DataFrames
# left_on='id' tells pandas to use the 'id' column from the left DataFrame (selected_df)
# right_on='player_id' tells it to use the 'player_id' column from the right DataFrame (player_info_df)
merged_df = pd.merge(selected_df, player_info_df, left_on='id', right_on='player_id')

print("Successfully merged player stats with player information.")
merged_df.head()


# Drop the original 'id' column from the DataFrame
merged_df.drop(columns=['player_id'], inplace=True)

# Optional: You can also rename 'player_id' to 'id' if you prefer a shorter name
merged_df.rename(columns={'id': 'player_id'}, inplace=True)

print("Cleaned up the ID columns.")
merged_df.head()

Successfully merged player stats with player information.
Cleaned up the ID columns.


,player_id,web_name,now_cost,selected_by_percent,form,minutes,total_points,bonus,bps,expected_goals_per_90,...,direct_freekicks_order,penalties_order,influence,creativity,threat,ict_index,status,chance_of_playing_next_round,position,team_code
0,1,Raya,5.5,19.9,10.0,90,10,3,38,0.00,...,0.0,0.0,49.2,0.0,0.0,4.9,a,100.0,Goalkeeper,3
1,16,Saka,10.0,17.0,0.0,1724,127,18,508,0.36,...,4.0,1.0,606.0,842.8,830.0,227.9,a,100.0,Midfielder,3
2,33,M.Bizot,4.5,0.3,0.0,0,0,0,0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,a,100.0,Goalkeeper,7
3,45,Sousa,4.0,0.3,0.0,0,0,0,0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,a,100.0,Defender,7
4,24,Nørgaard,5.5,0.5,0.0,0,0,0,0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,d,75.0,Midfielder,3


In [21]:
# --- Load Helper Files ---
# Teams data for team strength (CORRECTED URL)
teams_url = 'https://raw.githubusercontent.com/olbauday/FPL-Elo-Insights/main/data/2025-2026/teams.csv'
teams_df = pd.read_csv(teams_url)

# Gameweeks summary to find the next GW (CORRECTED FILENAME)
gameweeks_url = 'https://raw.githubusercontent.com/olbauday/FPL-Elo-Insights/main/data/2025-2026/gameweek_summaries.csv'
gameweeks_df = pd.read_csv(gameweeks_url)


# --- Step 1: Add Player's Own Team Strength ---
# Merge teams_df into your main DataFrame
final_df = pd.merge(merged_df, teams_df[['code', 'id', 'name', 'elo']], left_on='team_code', right_on='code')
final_df.drop(columns=['code'], inplace=True)
final_df.rename(columns={'id': 'team_id', 'name': 'team_name', 'elo': 'team_elo'}, inplace=True)


# --- Step 2: Find Next GW and Load Correct Fixtures ---
# Find the row where 'is_next' is True and get the 'id' (gameweek number)
next_gw = gameweeks_df[gameweeks_df['is_next'] == True].iloc[0]['id']
print(f"Next gameweek is GW{next_gw}. Fetching fixtures...")

# Construct the correct URL for the next gameweek's fixtures
fixtures_url = f'https://raw.githubusercontent.com/olbauday/FPL-Elo-Insights/main/data/2025-2026/By%20Gameweek/GW{next_gw}/fixtures.csv'
next_gw_fixtures = pd.read_csv(fixtures_url)


# --- Step 3: Add Opponent Info ---
def get_opponent_info(team_id, fixtures):
    match = fixtures[(fixtures['home_team'] == team_id) | (fixtures['away_team'] == team_id)]
    if not match.empty:
        if match.iloc[0]['home_team'] == team_id:
            return match.iloc[0]['away_team'], match.iloc[0]['away_team_elo'], True # Home
        else:
            return match.iloc[0]['home_team'], match.iloc[0]['home_team_elo'], False # Away
    return None, None, None

opponent_info = final_df['team_id'].apply(lambda x: get_opponent_info(x, next_gw_fixtures))
final_df[['opponent_team_id', 'opponent_elo', 'is_home']] = pd.DataFrame(opponent_info.tolist(), index=final_df.index)


# --- Step 4: Create Final 'elo_diff' Feature ---
final_df['elo_diff'] = final_df['team_elo'] - final_df['opponent_elo']

print("Opponent information and final features added.")
final_df[['web_name', 'team_name', 'team_elo', 'opponent_elo', 'elo_diff', 'is_home']].head()

Next gameweek is GW2. Fetching fixtures...
Opponent information and final features added.


,web_name,team_name,team_elo,opponent_elo,elo_diff,is_home
0,Raya,Arsenal,1998,1781.72,216.28,False
1,Saka,Arsenal,1998,1781.72,216.28,False
2,M.Bizot,Aston Villa,1871,1993.34,-122.34,False
3,Sousa,Aston Villa,1871,1993.34,-122.34,False
4,Nørgaard,Arsenal,1998,1781.72,216.28,False


In [22]:
final_df.head()

# Get the list of all column names from your final DataFrame
all_columns = list(final_df.columns)

# Print the total number of columns
print(f"You have a total of {len(all_columns)} columns in your final_df.")

# Print each column name, one per line, for easy reading
print("\n--- Column List ---")
for column in all_columns:
    print(column)

You have a total of 34 columns in your final_df.

--- Column List ---
player_id
web_name
now_cost
selected_by_percent
form
minutes
total_points
bonus
bps
expected_goals_per_90
expected_assists_per_90
expected_goal_involvements_per_90
expected_goals_conceded_per_90
starts_per_90
clean_sheets_per_90
saves_per_90
corners_and_indirect_freekicks_order
direct_freekicks_order
penalties_order
influence
creativity
threat
ict_index
status
chance_of_playing_next_round
position
team_code
team_id
team_name
team_elo
opponent_team_id
opponent_elo
is_home
elo_diff


In [34]:
# --- Step 1: Create a Clean, De-duplicated DataFrame for the Current Season ---
# Drop duplicates based on 'player_id' to get a unique entry for each active player
cleaned_prediction_df = final_df.drop_duplicates('player_id', keep='first')


# --- Step 2: Load and Prepare Last Season's Data ---
last_season_final_stats_url = 'https://raw.githubusercontent.com/olbauday/FPL-Elo-Insights/main/data/2024-2025/playerstats/playerstats.csv'
last_season_df = pd.read_csv(last_season_final_stats_url)

last_season_players_url = 'https://raw.githubusercontent.com/olbauday/FPL-Elo-Insights/main/data/2024-2025/players/players.csv'
last_season_players_df = pd.read_csv(last_season_players_url)

last_season_final_df = last_season_df[last_season_df['gw'] == 38]
last_season_final_with_names = pd.merge(
    last_season_final_df,
    last_season_players_df[['player_id', 'web_name']],
    left_on='id',
    right_on='player_id'
)[['web_name', 'points_per_game']]
last_season_final_with_names.rename(columns={'points_per_game': 'ppg_last_season'}, inplace=True)


# --- Step 3: Merge Historical Data into the Cleaned DataFrame ---
prediction_df_with_history = pd.merge(
    cleaned_prediction_df,
    last_season_final_with_names,
    on='web_name',
    how='left'
)
prediction_df_with_history['ppg_last_season'] = prediction_df_with_history['ppg_last_season'].fillna(2.0)


# --- Step 4: Calculate the DYNAMIC Blended Form ---

# First, get the current gameweek number
# (Assuming you have already loaded gameweeks_df and found the next_gw)
current_gw = next_gw 

# Define the dynamic weights based on the current gameweek
if current_gw <= 5:
    weight_last_season = 0.7
    weight_current_season = 0.3
elif 6 <= current_gw <= 10:
    # Linearly decrease the weight for last season
    weight_last_season = 0.7 - (current_gw - 5) * 0.1  # Decreases by 0.1 each GW
    weight_current_season = 1.0 - weight_last_season
else: # GW 11 and beyond
    weight_last_season = 0.1
    weight_current_season = 0.9

print(f"For GW{current_gw}, using weights: Last Season={weight_last_season:.1f}, Current Season={weight_current_season:.1f}")

# Calculate the blended form with the new dynamic weights
prediction_df_with_history['blended_form'] = \
    (weight_last_season * prediction_df_with_history['ppg_last_season']) + \
    (weight_current_season * prediction_df_with_history['form'].astype(float))

print("\n--- The 'Raya' entry with DYNAMICALLY blended form ---")
print(prediction_df_with_history[prediction_df_with_history['web_name'] == 'Raya'][['web_name', 'form', 'ppg_last_season', 'blended_form']])

For GW2, using weights: Last Season=0.7, Current Season=0.3

--- The 'Raya' entry with DYNAMICALLY blended form ---
  web_name  form  ppg_last_season  blended_form
0     Raya  10.0              3.7          5.59
